Smt modulo convex
https://people.eecs.berkeley.edu/~sseshia/pubdir/hscc17-smc.pdf

http://www.cyphylab.ee.ucla.edu/software - z3 + cplex

What about KiSSat + JuMP

There was that example solver from the paper. It was made by gluing together z3 and cplex.

it sure seems like we could plug together math opt interface and z3

Or maybe eventually picosay/ kissat/ cadical/ boolector

So what happens:
SAt solver returns opaque

We need to query mathopt for feasibility

upon failing to be feasible, we need a small inferred predicate to assert.



Zonotope = Affine domain
First order taylor expansion.
So Intervals + automatic differentiation are sufficient?
Could use exact interval arithemtic and collect all the rrors in single term
Nonlinear computations don't have such a bad increase of error.


Join and mee

Join is "easy"
meet is harder

Both suggest zonotope is a generator representation

http://www.lix.polytechnique.fr/~putot/Publications/cav09_taylor.pdf
http://vmcaischool19.tecnico.ulisboa.pt/~vmcaischool19.daemon/wp/wordpress/wp-content/uploads/2019/01/slidesVMCAI2019.pdf
Fluctuat analyzer

Satisfiability module JuMP

With the metatheory egraph, now we really have the core of an smt solver available.
Especially if I can build this explain operation.

EUF and SAT.

While I want human readable proofs from metatheory. Maybe the way to do that is extract the explanation
Could one then interpret the explanation in order of application and orientation using the classical rewrite system?



metatheory also provides rewrite rules, which are the seeds of the preoprocessing stage


If we pulled together all the corners of the julia ecosystem, we have a very feature complete SMT solver.
Intervals
groebner and algebra.
Homotopy conitnuation
Multiprecision stuff
algerbaic numbers?


That one interface. What was it called? ipasir interface
https://github.com/biotomas/ipasir
decsription of interfaxce
https://github.com/arminbiere/cadical/blob/master/src/cadical.hpp

What about just equality? Give me a miniamal subset of 

MicroSMT - just raw union find and SAT

linear equations and SAT?

So if I wanted to use ipasir, I should try just writing a C program first.
Leaping to FFI is insane.
https://github.com/JuliaInterop/Clang.jl/issues clang autogenerates api. Seems like it is working on a big update, so main package is dusty



In [2]:
using Pkg
Pkg.add("PicoSAT")


   Resolving package versions...
   Installed Qt_jll ─ v5.15.2+3
    Updating `~/.julia/environments/v1.6/Project.toml`
  [ff2beb65] + PicoSAT v0.3.3
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b99e7846] + BinaryProvider v0.5.10
  [ff2beb65] + PicoSAT v0.3.3
  [ede63266] ↑ Qt_jll v5.15.2+2 ⇒ v5.15.2+3
Precompiling project...
  ✓ Qt_jll
  ✓ GR_jll
  ✓ BinaryProvider
  ✓ PicoSAT
  ✓ GR
  ✓ Plots
6 dependencies successfully precompiled in 40 seconds (186 already precompiled)


In [4]:
using PicoSAT
cnf = Any[[1, -5, 4], [-1, 5, 3, 4], [-3, -4]];
PicoSAT.itersolve(cnf)
for sol in PicoSAT.itersolve(cnf)
           println(sol)
end

[1, -2, -3, -4, 5]
[1, -2, -3, 4, -5]
[1, -2, -3, 4, 5]
[1, -2, 3, -4, -5]
[1, -2, 3, -4, 5]
[1, 2, 3, -4, -5]
[1, 2, 3, -4, 5]
[1, 2, -3, -4, 5]
[1, 2, -3, 4, -5]
[1, 2, -3, 4, 5]
[-1, 2, -3, 4, -5]
[-1, 2, -3, 4, 5]
[-1, 2, -3, -4, -5]
[-1, 2, 3, -4, -5]
[-1, -2, 3, -4, -5]
[-1, -2, -3, -4, -5]
[-1, -2, -3, 4, 5]
[-1, -2, -3, 4, -5]


In [6]:
using Pkg
Pkg.add("Dualization")

  Resolving package versions...
  Installed Dualization ─ v0.3.3
Updating `~/.julia/environments/v1.5/Project.toml`
  [191a621a] + Dualization v0.3.3
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [191a621a] + Dualization v0.3.3


In [5]:
using PyCall
z3 = pyimport("z3")
q = z3.Bool("q")

PyObject q

It might be worthwhile just to use z3 to start. I know it, and Picosat isn't going to be that pleasant to wrko with, and I should be using boolector / kissat anyhow.

https://github.com/jump-dev/Dualization.jl

In [ ]:
using JuMP
using GLPK
using Dualization

model = Model(GLPK.Optimizer)



@objective(model, Min, 5x + 3 * y)


In [3]:
using JuMP
using GLPK
using Dualization

# Solving a problem the standard way
model = Model(dual_optimizer(GLPK.Optimizer))
@variable(model, x)
@variable(model, y)
@variable(model, z)
@constraint(model, con1, x + y >= 1)
@constraint(model, con2, x >= 1)
@objective(model, Min, 0)
dualize(model)

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [ ]:
struct Model
    linConstraints :: 

end